In [1]:
#STEP 1

#Importazione delle librerie necessarie al corretto funzionamento dello script

import os
import mimetypes
import csv

#Definizione delle funzioni che mi serviranno per eseguire lo script
#Ho scelto di affrontare in questo modo lo script in modo da avere
#un approccio modulare.

#Definisco il dizionario in cui associo la chiave (il tipo di file)
#con il valore (il nome che avranno le cartelle di destinazione)

def carica_dizionario():
    return {
        'audio': 'audio',
        'image': 'image',
        'text': 'docs'
    }

#Definisco la funzione prepara_ambiente che mi occorrà per ordinare
#i file dentro il percorso e gestire il percorso del file recap csv

def prepara_ambiente(files_path, recap_input):
    files_list = sorted(os.listdir(files_path))
    csv_file_path = os.path.join(files_path, recap_input)
    return files_list, csv_file_path

#Definisco la funzione per creare le cartelle per ogni chiave
#del dizionario nel caso in cui ancora non esista la cartella

def crea_cartelle_destinazione(files_path, dictionary):
    for key in dictionary.values():
        full_path = os.path.join(files_path, key)
        if not os.path.exists(full_path):
            os.mkdir(full_path)  
            
#Definisco una funzione che si occupa di riconoscere il file sulla base
#della sua estensione, grazie alla libreria mimetypes
            
def riconosci_tipo_file(file, dictionary):
    tipo_mime, _ = mimetypes.guess_type(file)
    for key, _ in dictionary.items():
        if tipo_mime and key in tipo_mime:
            return key
    return None

#Definisco la funzione che si occupa di spostare il file e stampa il recap,
#per operare lo spostamento utilizzo la funzione riconosci_tipo_file precedentemente
#definita. Una volta spostato il file la funzione scrive nel recap.csv (che deve esistere)
#le informazioni del file spostato

def sposta_file(file, files_path, dictionary, csv_writer):
    if file == 'recap.csv':
        return
    
    tipo_file = riconosci_tipo_file(file, dictionary)
    if tipo_file:
        dest_folder_path = os.path.join(files_path, dictionary[tipo_file])
        dest_file_path = os.path.join(dest_folder_path, file)
        src_file_path = os.path.join(files_path, file)
        
        os.rename(src_file_path, dest_file_path)
        
        name, extension = os.path.splitext(file)
        size = os.path.getsize(dest_file_path)
        
        csv_writer.writerow([name, tipo_file, size])
        print(f"{name} type:{tipo_file} size: {size}B")
        
#La funzione principale passa i vari argomenti alle funzioni precedentemente create, oltre
#che andare a creare o aggiornare (se già esiste) il file recap.csv.

def main():
    files_path = os.path.join('files')
    dictionary = carica_dizionario()
    recap_input = os.path.join('recap.csv')
    files_list, csv_file_path = prepara_ambiente(files_path, recap_input)
    crea_cartelle_destinazione(files_path, dictionary)
    file_exists = os.path.exists(csv_file_path)

    with open(csv_file_path, mode='a' if file_exists else 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            
            if not file_exists:
                csv_writer.writerow(['name', 'tipo_file', 'size'])

            for file in files_list:
                sposta_file(file, files_path, dictionary, csv_writer)

if __name__ == "__main__":
    main()

bw type:image size: 94926B
ciao type:text size: 12B
daffodil type:image size: 24657B
eclipse type:image size: 64243B
pippo type:text size: 8299B
song1 type:audio size: 1087849B
song2 type:audio size: 764176B
trump type:image size: 10195B


In [1]:
#STEP 2

#Importazione delle librerie necessarie al corretto funzionamento dello script
#(Avrei potuto inserire in un blocco a parte tutte le librerie, comprese quelle in 
#comune tra i 2 script. Le ho separate solo per aiutare nella comprensione del
#lavoro svolto sul singolo script.

import os
from PIL import Image
import numpy as np
import mimetypes
from tabulate import tabulate

#Definizione delle funzioni che mi serviranno per eseguire lo script
#Ho scelto di affrontare in questo modo lo script in modo da avere
#un approccio modulare.

#Definisco una funzione che si occupa di iterare e restituire i path dei vari file
#presenti dentro il path fornito

def carica_immagine(file_path_2):
    file_list_2 = sorted(os.listdir(file_path_2))
    image_paths = []
    
    for file in file_list_2:
        image_path = os.path.join(file_path_2, file)
        image_paths.append(image_path)
    
    return image_paths

#Definisco la funzione che si occuperà di classificare e recuperare
#sulla base del numero di ndim i file che passano all'interno del ciclo if.
#All'interno della funzione faccio in modo che vengano iterati 
#solamente i file immagine e non altri tipi di file, restituendo le informazioni
#utili alla risoluzione dell'esercizio.
#Recupero altezza e larghezza con la shape dell'immagine, mentre sfruttando ndim e shape
#recupero le informazioni di grayscale, R, G, B, e ALPHA

def estrai_info(image_paths):
    results = []
    
    for image_path in image_paths:
        tipo_mime, _ = mimetypes.guess_type(image_path)
    
        if not tipo_mime or not tipo_mime.startswith('image'):
            continue     

        with Image.open(image_path) as image:
            image_array = np.array(image)
            height, width = image_array.shape[:2]
            nome_file_con_estensione = os.path.basename(image_path)
            name, _ = os.path.splitext(nome_file_con_estensione)
            
            grayscale = R = G = B = A = None

            if image_array.ndim == 2:
                grayscale = image_array.mean().round(2)
            elif image_array.ndim == 3:
                if image_array.shape[2] == 4:
                    R, G, B, A = image_array.mean(axis=(0, 1)).round(2)
                elif image_array.shape[2] == 3:      
                    R, G, B = image_array.mean(axis=(0, 1)).round(2)
            else: 
                pass
                
            results.append((name, height, width, grayscale, R, G, B, A))

    return results

#Definisco la funzione principale in cui definisco l'header, gestisco i valori nulli
#e stampo la tabella 

def main():
    file_path_2 = os.path.join('files/image')
    image_paths = carica_immagine(file_path_2)
    results = estrai_info(image_paths)
    header = ['name', 'height', 'width', 'grayscale', 'R', 'G', 'B', 'ALPHA']
    print(tabulate(results, headers=header, tablefmt='fancy_grid', missingval='0.00'))
    
if __name__ == "__main__":
    main()

╒══════════╤══════════╤═════════╤═════════════╤════════╤════════╤═══════╤═════════╕
│ name     │   height │   width │   grayscale │      R │      G │     B │   ALPHA │
╞══════════╪══════════╪═════════╪═════════════╪════════╪════════╪═══════╪═════════╡
│ bw       │      512 │     512 │       21.48 │   0.00 │   0.00 │  0.00 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ daffodil │      500 │     335 │        0.00 │ 109.25 │  85.56 │  4.97 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ eclipse  │      256 │     256 │        0.00 │ 109.05 │ 109.52 │ 39.85 │  133.59 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ trump    │      183 │     275 │        0.00 │  97.01 │  98.99 │ 90.92 │    0.00 │
╘══════════╧══════════╧═════════╧═════════════╧════════╧════════╧═══════╧═════════╛
